In [1]:
import pandas as pd 
import geopandas as gpd

In [2]:
#read the GeoJson file
census_file = 'https://raw.githubusercontent.com/imadahmad97/EDA-of-Airbnb-Data/main/DataSets/GEOJSON%20Files/communes-75-paris.geojson'
cendf = gpd.read_file(census_file)

# Let's index by district name
cendf.set_index('nom', inplace=True)
cendf_2 = cendf

# Filter cleaned file to only Paris
paris_df_map = airbnb_raw_plus_review[(airbnb_raw_plus_review['city']=='Paris')].reset_index()

#combine the long and lat as coordinates
paris_df_map['coordinates'] = list(zip(paris_df_map["longitude"], paris_df_map["latitude"]))
paris_df_map['coordinates'] = paris_df_map['coordinates'].apply(Point)
locsdf = gpd.GeoDataFrame(paris_df_map, geometry='coordinates')
locsdf = locsdf.set_crs(epsg=4326) 

output_dict=dict() 
coordinates = paris_df_map['coordinates'].tolist()
count_of_review = paris_df_map['review_id_distinct_count'].tolist()
geo = cendf_2['geometry'].tolist()
district_code = cendf_2['code'].tolist()
output_dict=dict() 
for j in range(len(geo)):
    for i in range(len(coordinates)):
        if geo[j].contains(coordinates[i]):
            if district_code[j] in output_dict:
                if not (np.isnan(count_of_review[i])):
                    output_dict[district_code[j]]+=count_of_review[i]
            else:
                if not (np.isnan(count_of_review[i])):
                    output_dict[district_code[j]]=count_of_review[i]

output_dict_paris = output_dict

#turn the dictionary to dataframe, then left join to the cendf file
output_dict_df = pd.DataFrame(output_dict_paris.items(), columns=['code', 'review_count'])
cendf_2['nom'] = cendf_2.index
cendf_3 = pd.merge(cendf_2, output_dict_df , on='code', how='left')
cendf_3.set_index('nom', inplace=True)

# Choropleth map of calgary resident count via px.choropleth_mapbox()
cendf_4 = cendf_3.to_crs(epsg=4326)

# plot the map
fig = px.choropleth_mapbox(cendf_4, geojson=cendf_4,
                           locations=cendf_4.index, 
                           color="review_count",
                           color_continuous_scale = 'YlGn',
                           center={"lat": 48.8566, "lon": 2.3522}, # Paris
                           mapbox_style="carto-positron",
                           opacity=0.75,
                           zoom=10, 
                           title = 'Paris Review Density')

fig.update_layout(margin={"r":50,"t":50,"l":50,"b":50}, 
                  autosize=True,
                  height=600 )

fig.show()

NameError: name 'airbnb_raw_plus_review' is not defined